In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import nltk
import sklearn
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import torch
print(torch.__version__)
from transformers import pipeline

# # Download latest version
# path = kagglehub.dataset_download("eshummalik/socialbuzz-sentiment-analytics")

# print("Path to dataset files:", path)

2.10.0+cpu


In [3]:
path = r"C:\Users\Kelby Mace\.cache\kagglehub\datasets\eshummalik\socialbuzz-sentiment-analytics\versions\4"

df = pd.read_csv(os.path.join(path, "sentimentdataset.csv"))

# Fix data tytpes
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Likes'] = df['Likes'].astype(int)
df['Retweets'] = df['Retweets'].astype(int)

df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [5]:
# Clean text

def clean_text(text):
    """Lowercase, strip whitespace, remove punctuation, and replace numbers with <NUM>."""
    text = text.lower()
    text = text.strip()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'[0-9]', '<NUM>', text)
    return text


# Apply text cleaning function
df['Text'] = df['Text'].apply(clean_text)
df['Sentiment'] = df['Sentiment'].apply(lambda x: x.strip().lower())

#Add text length feature
df['Text Length'] = df['Text'].apply(len)

# Drop extra index columns
if 'Unnamed: 0.1' in df.columns:
    df = df.drop(columns=['Unnamed: 0.1'])
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12,36
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8,33
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15,32
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18,42
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19,42


In [25]:
# df.hist(column='Text Length', by='Sentiment', bins=50,figsize=(12,4))

In [6]:
df['Sentiment'].value_counts()

Sentiment
positive          45
joy               44
excitement        37
contentment       19
neutral           18
                  ..
lostlove           1
emotionalstorm     1
suffering          1
bittersweet        1
intrigue           1
Name: count, Length: 191, dtype: int64

In [7]:
candidate = ["positive", "neutral", "negative"]
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def map_label(label: str):
    out = classifier(label, candidate_labels=candidate, multi_label=False)
    return out["labels"][0], out["scores"][0]  # predicted class + confidence

# Example
map_label("furious")


config.json: 0.00B [00:00, ?B/s]

c:\Users\Kelby Mace\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kelby Mace\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
'The read operation timed out' thrown while requesting GET https://huggingface.

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

('negative', 0.983331024646759)

In [ ]:
# Make new label column from zero-shot classification with fewer categories

# df['Predicted Sentiment'] = df['Text'].apply(lambda x: map_label(x)[0])
# df['Prediction Confidence'] = df['Text'].apply(lambda x: map_label(x)[1])

df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12,36,positive,0.936028
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8,33,negative,0.976853
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15,32,positive,0.923207
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18,42,positive,0.975727
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19,42,positive,0.662256


In [ ]:
# df.to_csv("sentiment_with_predictions.csv", index=False)

In [16]:
df['Predicted Sentiment'].value_counts(normalize=True)

Predicted Sentiment
positive    0.684426
negative    0.278689
neutral     0.036885
Name: proportion, dtype: float64

In [18]:
df.loc[df['Prediction Confidence'] == df['Prediction Confidence'].min()]

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence
349,contemplating lifes mysteries under the starry...,contemplation,2016-07-02 23:30:00,Stargazer,Twitter,#Contemplation #StarryNight,28,55,UK,2016,7,2,23,56,negative,0.348642


In [ ]:
# df['Sentiment Reduced'], df['Sentiment Reduced Confidence'] = zip(*df['Sentiment'].apply(lambda x: map_label(x)))

In [22]:
df.to_csv("sentiment_with_predictions.csv", index=False)

In [23]:
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence,Sentiment Reduced,Sentiment Reduced Confidence
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12,36,positive,0.936028,positive,0.987529
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8,33,negative,0.976853,negative,0.988335
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15,32,positive,0.923207,positive,0.987529
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18,42,positive,0.975727,positive,0.987529
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19,42,positive,0.662256,neutral,0.855826


In [ ]:
# For sentiments already classified as positive, neutral, or negative, how often does the predicted sentiment match the original sentiment?

df_three = df.copy()

df_three = df_three.loc[df_three['Sentiment'].isin(["positive", "neutral", "negative"])]
df_three['Text Sentiment Match'] = df_three.apply(lambda row: row['Sentiment'] == row['Predicted Sentiment'], axis=1)

# This works 100% of the time -- good gut check
df_three['Sentiment Prediction Match'] = df_three.apply(lambda row: row['Sentiment'] == row['Sentiment Reduced'], axis=1)

df_three['Predictions Match'] = df_three.apply(lambda row: row['Text Sentiment Match'] == row['Sentiment Prediction Match'], axis=1)

df_three['Text Sentiment Match'].value_counts(normalize=True), df_three['Sentiment Prediction Match'].value_counts(normalize=True), df_three['Predictions Match'].value_counts(normalize=True)

(Text Sentiment Match
 True     0.671642
 False    0.328358
 Name: proportion, dtype: float64,
 Sentiment Prediction Match
 True    1.0
 Name: proportion, dtype: float64,
 Predictions Match
 True     0.671642
 False    0.328358
 Name: proportion, dtype: float64)